So im going to need to reduce the size of the enron emails to 500 mb

In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub langchain -qU langchain-google-genai pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Enron/emails.csv')

In [ ]:
data

,message
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)\nFrom: p..."
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)\nFrom: ph..."
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)\nFrom: p..."
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)\nFrom: p..."
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)\nFrom: p..."
...,...
517396,"Wed, 28 Nov 2001 13:30:11 -0800 (PST)\nFrom: j..."
517397,"Wed, 28 Nov 2001 12:47:48 -0800 (PST)\nFrom: j..."
517398,"Wed, 28 Nov 2001 07:20:00 -0800 (PST)\nFrom: j..."
517399,"Tue, 27 Nov 2001 11:52:45 -0800 (PST)\nFrom: j..."


In [ ]:
# !pip install cuml-cuda11 --extra-index-url=https://rapidsai.github.io/rapidsai-csp/colab  # Adjust based on your CUDA version
# !pip install joblib pandas

import pandas as pd
import numpy as np
import joblib
from cuml.manifold import UMAP

# Load your embedding dataset from a CSV file.
# The dataset should have 384 columns of embedding values.
input_csv_path = "/content/drive/MyDrive/Enron/embeddings.csv"  # update path accordingly
df = pd.read_csv(input_csv_path)

# Convert DataFrame to a NumPy array and ensure it's float32 (for GPU compatibility)
X = df.values.astype(np.float32)

# Initialize UMAP for GPU with target dimensionality of 80.
reducer = UMAP(n_components=80, random_state=42)

# Fit UMAP on your data and transform it
X_reduced = reducer.fit_transform(X)

# Save the reduced embeddings to a new CSV (optional)
output_csv_path = "/content/drive/MyDrive/reduced_embeddings.csv"
pd.DataFrame(X_reduced, columns=[f"umap_{i+1}" for i in range(80)]).to_csv(output_csv_path, index=False)
print(f"Reduced embeddings saved to {output_csv_path}")

# Save the fitted UMAP model using joblib
joblib_file = "/content/drive/MyDrive/Enron/umap_model.joblib"
joblib.dump(reducer, joblib_file)
print(f"Fitted UMAP model saved to {joblib_file}")


[2025-03-28 17:49:38.977] [CUML] [info] build_algo set to brute_force_knn because random_state is given
Reduced embeddings saved to /content/drive/MyDrive/reduced_embeddings.csv
Fitted UMAP model saved to /content/drive/MyDrive/Enron/umap_model.joblib


In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from google.colab import drive
from cuml.manifold import UMAP
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from urllib.parse import quote_plus
from pymongo import MongoClient
import joblib

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Enron/Emails/emails.csv')

In [ ]:
def compute_embeddings(texts):
    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    if isinstance(texts, str):
        texts = [texts]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_name = "intfloat/e5-small-v2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    input_texts = [f"query: {text}" for text in texts]
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    sentence_embeddings = mean_pooling(outputs, inputs['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    embeddings = sentence_embeddings.cpu().numpy()
    if len(texts) == 1:
        return embeddings[0]
    else:
        return embeddings

def reduce_embedding(embedding_array, target_dimensions=80):
    if not isinstance(embedding_array, np.ndarray):
        raise ValueError("Input should be a NumPy array.")
    if embedding_array.ndim == 1:
        embedding_array = embedding_array.astype(np.float32).reshape(1, -1)
    else:
        embedding_array = embedding_array.astype(np.float32)
    reducer = joblib.load('/content/drive/MyDrive/Enron/umap_model.joblib')
    reduced_embeddings = reducer.transform(embedding_array)
    return reduced_embeddings

def generate_alternative_questions(question: str) -> list:
    template = (
        "You are an AI language model assistant. Your task is to generate five different "
        "versions of the given user question. Provide these alternative questions separated by newlines. "
        "Original question: {question}"
    )
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    gemini_llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        google_api_key="API_KEY",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
    generate_queries = (
        prompt_perspectives
        | gemini_llm
        | StrOutputParser()
        | (lambda x: [item for item in x.split("\n") if item.strip()])
    )
    result = generate_queries.invoke({"question": question})
    return result

def generate_sub_questions(question: str) -> list:
    template = (
        """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (5 queries):"""
    )
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    gemini_llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        google_api_key="API_KEY",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
    generate_queries = (
        prompt_perspectives
        | gemini_llm
        | StrOutputParser()
        | (lambda x: [item for item in x.split("\n") if item.strip()])
    )
    result = generate_queries.invoke({"question": question})
    return result

def generate_step_back_question(question: str) -> str:
    examples = [
        {
            "input": "Could the members of The Police perform lawful arrests?",
            "output": "what can the members of The Police do?",
        },
        {
            "input": "Jan Sindel’s was born in what country?",
            "output": "what is Jan Sindel’s personal history?",
        },
    ]
    example_prompt = ChatPromptTemplate.from_messages([
        ("human", "{input}"),
        ("ai", "{output}")
    ])
    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=examples,
    )
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:"),
        few_shot_prompt,
        ("user", "{question}")
    ])
    gemini_llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        google_api_key="API_KEY",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )
    generate_query_chain = prompt | gemini_llm | StrOutputParser()
    result = generate_query_chain.invoke({"question": question})
    return result

def vector_search(query_vector) -> list:
    username = "username"
    password = "pass@123"
    database_name = "dbname"
    collection_name = "colname"
    encoded_username = quote_plus(username)
    encoded_password = quote_plus(password)
    mongo_uri = (
        f"mongodb+srv://{encoded_username}:{encoded_password}"
        "@enroncluster.gmdslts.mongodb.net/?retryWrites=true&w=majority&appName=enroncluster"
    )
    os.environ['MONGO_URI'] = mongo_uri
    mongo_uri_env = os.environ.get('MONGO_URI')
    client = MongoClient(mongo_uri_env)
    db = client[database_name]
    collection = db[collection_name]
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1, -1)
    nested_results = []
    for vec in query_vector:
        pipeline = [
            {
                "$vectorSearch": {
                    "queryVector": vec.tolist(),
                    "path": "embedding",
                    "numCandidates": 100,
                    "limit": 4,
                    "index": "enronsearch"
                }
            },
            {
                "$project": {"id": 1, "_id": 0}
            }
        ]
        results_cursor = collection.aggregate(pipeline)
        results = list(results_cursor)
        nested_results.append(results)
    client.close()
    return nested_results

def get_emails(results):
    df = pd.read_csv('/content/drive/MyDrive/Enron/emails.csv')
    all_emails = []
    for sublist in results:
        line_emails = []
        for res in sublist:
            row_idx = res.get('id')
            try:
                email_val = df.iloc[row_idx]['message']
            except IndexError:
                email_val = None
            line_emails.append(email_val)
        all_emails.append(line_emails)
    return all_emails

def ask_question_with_context(question: str, nested_emails: list) -> str:
    flattened_emails = [email for sublist in nested_emails for email in sublist if email]
    context = "\n\n".join(flattened_emails)
    template = """Answer the following question based on this context:

{context}

Question: {question}
"""
    prompt = ChatPromptTemplate.from_template(template)
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-pro",
        google_api_key="API_KEY",
        temperature=0,
        max_tokens=1000,
        timeout=None,
        max_retries=2,
    )
    chain = prompt | llm | StrOutputParser()
    output = chain.invoke({"context": context, "question": question})
    return output


In [ ]:
query = "when was enron in financial trouble"
step_back_question = generate_step_back_question(query)
embedded_query = compute_embeddings(step_back_question)
reduced_query = reduce_embedding(embedded_query)
results = vector_search(reduced_query)
emails = get_emails(results)
answers = ask_question_with_context(query, emails)
print(answers)